# 0. Install and Import Dependencies

In [ ]:
# !pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1 tensorflow-hub opencv-python matplotlib

In [4]:
import tensorflow as tf
import tensorflow_hub as hub
import cv2
from matplotlib import pyplot as plt
import numpy as np

In [5]:
# Optional if you are using a GPU
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

# 1. Load Model

In [6]:
model = hub.load('https://tfhub.dev/google/movenet/multipose/lightning/1')
movenet = model.signatures['serving_default']

# 2. Make Detections

In [9]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    
    # Resize image
    img = frame.copy()
    img = tf.image.resize_with_pad(tf.expand_dims(img, axis=0), 384,640)
    input_img = tf.cast(img, dtype=tf.int32)
    
    # Detection section
    results = movenet(input_img)
    keypoints_with_scores = results['output_0'].numpy()[:,:,:51].reshape((6,17,3))
    
    # Render keypoints 
    loop_through_people(frame, keypoints_with_scores, EDGES, 0.25)
    cv2.imshow('Movenet Multipose', frame)

    if cv2.waitKey(10) & 0xFF==ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

In [1]:
x=1
cap = cv2.imread(fr"F:\Ojas SSD\Ojas\Python\Scienceexib\Frames\frame{x}.jpg")
frame=cap
    
# Resize image
img = frame.copy()
img = tf.image.resize_with_pad(tf.expand_dims(img, axis=0), 384,640)
input_img = tf.cast(img, dtype=tf.int32)

# Detection section
results = movenet(input_img)
keypoints_with_scores = results['output_0'].numpy()[:,:,:51].reshape((6,17,3))

# Render keypoints 
loop_through_people(frame, keypoints_with_scores, EDGES, 0.22)
while True:
    cv2.imshow('Movenet Multipose', frame)
    if cv2.waitKey(10) & 0xFF==ord('q'):
        break
    if cv2.waitKey(10) & 0xFF==ord('n'):
        break
    
cv2.imwrite("Hey.png",frame)
cv2.destroyAllWindows()

NameError: name 'cv2' is not defined

In [10]:
keypoints_with_scores

array([[[0.27003023, 0.30028734, 0.11083874],
        [0.24188834, 0.32549033, 0.13397208],
        [0.20620915, 0.3096327 , 0.15522145],
        [0.24590789, 0.32197636, 0.1341855 ],
        [0.17734155, 0.27784932, 0.15706302],
        [0.31359065, 0.24004896, 0.10324387],
        [0.2573926 , 0.20443778, 0.09732514],
        [0.45238137, 0.25621995, 0.03155606],
        [0.42702138, 0.15575382, 0.02433665],
        [0.48738402, 0.24967527, 0.05952699],
        [0.48233688, 0.16621605, 0.02065668],
        [0.39090043, 0.10764223, 0.03569803],
        [0.35443968, 0.04233905, 0.03422426],
        [0.44158122, 0.1586245 , 0.09642186],
        [0.45153326, 0.14394687, 0.09243739],
        [0.44639644, 0.11272778, 0.02921812],
        [0.40804225, 0.07409511, 0.02091025]],

       [[0.15904874, 0.3768737 , 0.05138283],
        [0.13875605, 0.38250974, 0.0791804 ],
        [0.14922044, 0.37819988, 0.05803177],
        [0.12395483, 0.40287772, 0.05720044],
        [0.14968644, 0.3812305 ,

In [11]:
32*12

384

In [8]:
# Function to loop through each person detected and render
def loop_through_people(frame, keypoints_with_scores, edges, confidence_threshold):
    for person in keypoints_with_scores:
        draw_connections(frame, person, edges, confidence_threshold)
        draw_keypoints(frame, person, confidence_threshold)

# 3. Draw Keypoints

In [1]:
def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 6, (0,255,0), -1)

# 4. Draw Edges

In [2]:
EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

In [3]:
def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 4)